In [ ]:
import os
import json
import re
from groq import Groq

client = Groq(
    api_key="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",
)

# Modify the prompt to request JSON output
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": (
                "Explain the importance of fast language models. "
                "Provide the answer in JSON format with keys 'summary', 'benefits', and 'applications'."
            ),
        }
    ],
    model="llama3-8b-8192",
)

content = chat_completion.choices[0].message.content

# Use regex to extract JSON content in case there is extra text
json_match = re.search(r'\{.*\}', content, re.DOTALL)

if json_match:
    json_content = json_match.group()
    try:
        data = json.loads(json_content)
        print(json.dumps(data, indent=4))
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        print("Content was:", json_content)
else:
    print("No JSON content found in the response.")


In [ ]:
#Generate Keys

import requests

# Replace with your actual endpoint URL
url = "http://localhost:8000/generate_api_key"

# The master key from any service (ensure it's valid)
master_key = "your_master_service_api_key"  # Replace with the actual master key

# Prepare the request data
data = {
    "name": "Jane Doe",
    "email": "jane.doe@example.com",
    "privilege": "user",  # or "admin"
    "master_key": master_key
}

# Prepare headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, headers=headers, json=data)

# Handle the response
if response.status_code == 200:
    # Successful key generation
    result = response.json()
    print(f"Your new API key is: {result['api_key']}")
elif response.status_code == 400:
    # Bad request (e.g., invalid privilege level)
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 401:
    # Unauthorized (invalid master key)
    print(f"Error {response.status_code}: {response.json()['detail']}")
else:
    # Other errors
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#LLM Inference Default

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
    "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    "model": "llama3.1-8b",
    "service_name": "cerebras",
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#LLM Inference Custom Parameters

import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

url = "http://localhost:8000/chat/completions"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

data = {
    "messages": [
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    "model": "llama3.1-8b",
    "service_name": "cerebras",
    "temperature": 0.7,
    "max_tokens": 500,
    "top_p": 0.9,
    "stop": ["\n"]
}

response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
#Delete Key

import requests

# Replace with your actual endpoint URL
url = "http://localhost:8000/delete_api_key"

# The admin's email and API key
admin_email = "admin@example.com"
admin_api_key = "admin_api_key_value"

# The email of the user whose API key you want to delete
email_to_delete = "user@example.com"

# Prepare the request data
data = {
    "admin_email": admin_email,
    "admin_api_key": admin_api_key,
    "email_to_delete": email_to_delete
}

# Prepare headers
headers = {
    "Content-Type": "application/json"
}

# Send the DELETE request
response = requests.delete(url, headers=headers, json=data)

# Handle the response
if response.status_code == 200:
    # Successful deletion
    print(response.json())
elif response.status_code == 404:
    # Email to delete not found
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 401:
    # Invalid admin API key
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 403:
    # Admin privilege required
    print(f"Error {response.status_code}: {response.json()['detail']}")
else:
    # Other errors
    print(f"Error {response.status_code}: {response.text}")


In [ ]:
#Change privilege

import requests

# Replace with your actual endpoint URL
url = "http://localhost:8000/change_privilege"

# The admin's email and API key
admin_email = "admin@example.com"
admin_api_key = "admin_api_key_value"

# The email of the user whose privilege level you want to change
email_to_change = "user@example.com"

# The new privilege level ("user" or "admin")
new_privilege = "admin"

# Prepare the request data
data = {
    "admin_email": admin_email,
    "admin_api_key": admin_api_key,
    "email_to_change": email_to_change,
    "new_privilege": new_privilege
}

# Prepare headers
headers = {
    "Content-Type": "application/json"
}

# Send the PUT request
response = requests.put(url, headers=headers, json=data)

# Handle the response
if response.status_code == 200:
    # Successful privilege change
    print(response.json())
elif response.status_code == 404:
    # Email to change not found
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 401:
    # Invalid admin API key
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 403:
    # Admin privilege required
    print(f"Error {response.status_code}: {response.json()['detail']}")
elif response.status_code == 400:
    # Invalid privilege level
    print(f"Error {response.status_code}: {response.json()['detail']}")
else:
    # Other errors
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
import requests

api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key
url = "http://localhost:8000/chat/completions"  # Adjust the URL if necessary

# System prompt
system_prompt = """
You're evaluating writing style in text.
Your evaluations must always be in JSON format. Here is an example JSON response:

{ "readability": 4, "conciseness": 2 }

"""

# User prompt
tweet = "This is a sample text that needs evaluation."
prompt = f"""
Evaluate the text:

{tweet}

You're evaluating the readability and conciseness with values from 0 (extremely bad) to 10 (extremely good).
"""

# Messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
]

# Request data
data = {
    "messages": messages,
    "model": "llama3.1-8b",  # Replace with your desired model
    "service_name": "cerebras",     # Replace with your desired service
    "temperature": 0.0,         # Low temperature for deterministic output
    "max_tokens": 100,          # Adjust as needed
    "top_p": 1.0,
}

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Make the API call
response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    result = response.json()
    print(result["content"])
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:

model = ChatOllama(model="mistral:7b-instruct-v0.2-q4_1")
parser = StrOutputParser()
output = 'News_data_Company_SP500_3_Sentiment_Score.xlsx'
Question = """
    In this format respond to the news aricle given in the context interpret it and classify and respond in this JSON format only. No other things and extra explaination or interpretation.
    JSON = {
        Sentiment: Positive/Negative/Neutral,
        Corruption: Yes/No/Possible,
        Fraud: Yes/No/Possible,
        Type: (Type of news article),
        Explanation: (Write Explanation Here),
    }
    Return just the JSON No matter what. Strictly the formatted JSON.
"""

In [ ]:
def is_valid_json(json_data):
    try:
        json_object = json.loads(json_data)
    except ValueError as e:
        return False
    return True

def fix_json(json_data):
    try:
        if not json_data.strip().endswith('}'):
            json_data += '}'
        return json_data
    except Exception as e:
        return json_data
    
def perform_sentiment_analysis(text, model, parser, Question):
    template = """
    This is a news article from Thompson Reuters. Use this news article and respond only in the format asked in the question nothing extra. Do not respond with anything else just the JSON format.

    Context: {context}

    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | parser
    response = chain.invoke({
        "context": text,
        "question": Question
    })
    if not is_valid_json(response):
        fixed_response = fix_json(response)
        if is_valid_json(fixed_response):
            return json.loads(fixed_response)
        else:
            response = {
                'Sentiment': None,
                'Corruption': None,
                'Fraud': None,
                'Type': None,
                'Explanation': "Error in Analysis"
            }
            return(response)
    else:
        return json.loads(response)

In [ ]:
# AI Trading Agent for Mid-Frequency Trading on Apple Stock (AAPL)

import requests
import openbb
import pandas as pd
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},  # Stock positions
    'transaction_history': []  # Record of transactions
}

# Function to execute trades based on LLM recommendation
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['AAPL'] = portfolio['positions'].get('AAPL', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'AAPL' in portfolio['positions'] and portfolio['positions']['AAPL'] > 0:
            shares_to_sell = portfolio['positions']['AAPL']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['AAPL'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")

# Function to calculate portfolio value
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value

# Fetch historical stock data for AAPL
stock_data = openbb.stocks.load('AAPL', start_date='2022-01-01', end_date='2022-12-31')

# Ensure the data is sorted by date
stock_data = stock_data.sort_index()

# Reset index to have 'Date' as a column
stock_data = stock_data.reset_index()

# Loop over each day in the stock data
for index, row in stock_data.iterrows():
    # Prepare the prompt with recent data up to the current day
    recent_data = stock_data.loc[max(0, index - 4):index].to_string(index=False)
    date_str = row['Date'].strftime('%Y-%m-%d')
    prompt = f"""
You are a financial analyst. Based on the following recent stock data for Apple (AAPL), provide a buy, hold, or sell recommendation. Respond with only one word: buy, hold, or sell.

Stock Data:
{recent_data}

Recommendation:
"""
    # Call the LLM service
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-8b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        # Ensure the recommendation is valid
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        # Execute the trade
        current_price = row['Close']
        execute_trade(recommendation, current_price, date_str)
        # Optionally, pause or wait for the next time interval
        time.sleep(0.1)  # Small delay to simulate time between trades
    else:
        print(f"Error {response.status_code}: {response.text}")
        break  # Exit the loop on error

# Final portfolio value calculation
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

# Display final portfolio performance
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)

In [ ]:
!pip install openbb-terminal-sdk


In [ ]:
# Install the openbb-terminal-sdk package
# %pip install openbb-terminal-sdk

import openbb
print(openbb.__version__)

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},  # Stock positions
    'transaction_history': []  # Record of transactions
}

# Function to execute trades based on LLM recommendation
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['AAPL'] = portfolio['positions'].get('AAPL', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'AAPL' in portfolio['positions'] and portfolio['positions']['AAPL'] > 0:
            shares_to_sell = portfolio['positions']['AAPL']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['AAPL'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")

# Function to calculate portfolio value
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value

# Fetch historical stock data for AAPL using yfinance
stock_data = yf.download('AAPL', start='2022-01-01', end='2022-12-31')

# Ensure the data is sorted by date and reset index
stock_data = stock_data.sort_index().reset_index()

# Loop over each day in the stock data
for index, row in stock_data.iterrows():
    # Prepare the prompt with recent data up to the current day
    recent_data = stock_data.loc[max(0, index - 4):index].to_string(index=False)
    date_str = row['Date'].strftime('%Y-%m-%d')
    prompt = f"""
You are a financial analyst. Based on the following recent stock data for Apple (AAPL), provide a buy, hold, or sell recommendation. Respond with only one word: buy, hold, or sell.

Stock Data:
{recent_data}

Recommendation:
"""
    # Call the LLM service
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-8b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        # Ensure the recommendation is valid
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        # Execute the trade
        current_price = row['Close']
        execute_trade(recommendation, current_price, date_str)
        # Optionally, pause or wait for the next time interval
        time.sleep(0.1)  # Small delay to simulate time between trades
    else:
        print(f"Error {response.status_code}: {response.text}")
        break  # Exit the loop on error

# Final portfolio value calculation
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

# Display final portfolio performance
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},  # Stock positions
    'transaction_history': []  # Record of transactions
}

# Function to execute trades based on LLM recommendation
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['NVDA'] = portfolio['positions'].get('NVDA', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'NVDA' in portfolio['positions'] and portfolio['positions']['NVDA'] > 0:
            shares_to_sell = portfolio['positions']['NVDA']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['NVDA'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")

# Function to calculate portfolio value
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value

# Fetch historical stock data for NVDA using yfinance
stock_data = yf.download('NVDA', start='2023-01-01', end='2023-12-31')

# Ensure the data is sorted by date and reset index
stock_data = stock_data.sort_index().reset_index()

# Loop over each day in the stock data
for index, row in stock_data.iterrows():
    # Prepare the prompt with recent data up to the current day
    recent_data = stock_data.loc[max(0, index - 4):index].to_string(index=False)
    date_str = row['Date'].strftime('%Y-%m-%d')
    prompt = f"""
You are a financial analyst. Based on the following recent stock data for Nvidia (NVDA), provide a buy, hold, or sell recommendation. Respond with only one word: buy, hold, or sell.

Stock Data:
{recent_data}

Recommendation:
"""
    # Call the LLM service
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-8b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        # Ensure the recommendation is valid
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        # Execute the trade
        current_price = row['Close']
        execute_trade(recommendation, current_price, date_str)
        # Optionally, pause or wait for the next time interval
        time.sleep(0.1)  # Small delay to simulate time between trades
    else:
        print(f"Error {response.status_code}: {response.text}")
        break  # Exit the loop on error

# Final portfolio value calculation
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

# Display final portfolio performance
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)

# AI Trading Agent for Mid-Frequency Trading on Nvidia Stock (NVDA)

This code is a mid-frequency trading bot that uses an LLM (large language model) service to generate daily trading recommendations (buy, hold, sell) for Nvidia stock (NVDA). The bot simulates trading over a given historical period and tracks the portfolio's performance.

### Code Explanation

```python
import requests
import yfinance as yf
import pandas as pd
import time
```

- **Modules**: `requests` for API calls, `yfinance` for fetching historical stock data, `pandas` for data manipulation, and `time` for introducing delays in trading.

---

### 1. Setting Up the API Configuration for LLM Service

```python
api_key = "your_actual_api_key_here"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}
```

- **API Key**: Used to authenticate requests to the LLM service.
- **LLM Service Configuration**: Defines the URL and headers required to make a request to the LLM model, which will provide trading recommendations.

---

### 2. Initializing the Portfolio

```python
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},  # Stock positions
    'transaction_history': []  # Record of transactions
}
```

- **Portfolio Initialization**: Starts with `$10,000` in cash, no positions, and an empty transaction history.

---

### 3. Function to Execute Trades

```python
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['NVDA'] = portfolio['positions'].get('NVDA', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'NVDA' in portfolio['positions'] and portfolio['positions']['NVDA'] > 0:
            shares_to_sell = portfolio['positions']['NVDA']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['NVDA'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")
```

- **execute_trade**: Executes trades based on the LLM's recommendation:
    - **Buy**: Invests 10% of cash to buy shares at the current price.
    - **Sell**: Sells all NVDA shares if owned.
    - **Hold**: Does nothing if the recommendation is to hold.

---

### 4. Calculating the Portfolio Value

```python
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value
```

- **calculate_portfolio_value**: Computes the portfolio's total value by adding cash and the current market value of any stock positions.

---

### 5. Fetching Historical Data for NVDA

```python
stock_data = yf.download('NVDA', start='2023-01-01', end='2023-12-31')

# Ensure the data is sorted by date and reset index
stock_data = stock_data.sort_index().reset_index()
```

- **stock_data**: Downloads Nvidia stock data for 2023 using `yfinance`.
- **Data Preparation**: Sorts and resets the index, making the data easier to loop through by date.

---

### 6. Looping Over Each Day in the Stock Data

```python
for index, row in stock_data.iterrows():
    # Prepare the prompt with recent data up to the current day
    recent_data = stock_data.loc[max(0, index - 4):index].to_string(index=False)
    date_str = row['Date'].strftime('%Y-%m-%d')
    prompt = f"""
You are a financial analyst. Based on the following recent stock data for Nvidia (NVDA), provide a buy, hold, or sell recommendation. Respond with only one word: buy, hold, or sell.

Stock Data:
{recent_data}

Recommendation:
"""
```

- **Prompt Preparation**: Sends a 5-day window of recent stock data to the LLM for recommendation. 
- **Looping Through Each Day**: Loops over each row of historical data to fetch and analyze recent stock prices for decision-making.

---

### 7. Calling the LLM Service for Recommendation

```python
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-8b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
```

- **LLM Request**: Sends the prompt to the LLM with the required parameters to receive a trading recommendation (buy, hold, sell).

---

### 8. Executing the Trade

```python
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        current_price = row['Close']
        execute_trade(recommendation, current_price, date_str)
        time.sleep(0.1)
    else:
        print(f"Error {response.status_code}: {response.text}")
        break
```

- **Executing the Trade**: If the recommendation is valid, it executes the corresponding trade, with a short delay to simulate real-time trading.

---

### 9. Final Portfolio Value Calculation

```python
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100
```

- **Final Value Calculation**: Computes the total return based on the final portfolio value at the end of the period.

---

### 10. Displaying Final Portfolio Performance

```python
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)
```

- **Performance Display**: Outputs the final portfolio value, total return, remaining cash, stock positions, and transaction history.

---

### Summary

This code provides a simulated trading environment, utilizing an LLM to analyze recent stock trends and make trading decisions. It calculates the portfolio value and tracks the overall performance, making it suitable for exploring mid-frequency trading strategies using AI-driven recommendations.

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},
    'transaction_history': []
}

# Function to calculate indicators for quantitative analysis
def calculate_indicators(stock_data):
    # Moving Averages
    stock_data['MA20'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()
    
    # Bollinger Bands
    stock_data['BB_upper'] = stock_data['MA20'] + (stock_data['Close'].rolling(window=20).std() * 2)
    stock_data['BB_lower'] = stock_data['MA20'] - (stock_data['Close'].rolling(window=20).std() * 2)
    
    # Momentum
    stock_data['Momentum'] = stock_data['Close'] - stock_data['Close'].shift(4)
    
    return stock_data

# Speculative Decoding: Generate a speculative vector based on recent price changes
def speculative_decoding(stock_data, lookback=10):
    # Get the recent price changes as a vector
    recent_prices = stock_data['Close'].tail(lookback).values
    future_speculation = np.diff(recent_prices)  # Calculate hypothetical future change
    
    # Normalize the speculative vector
    spec_vector = future_speculation / np.linalg.norm(future_speculation) if np.linalg.norm(future_speculation) != 0 else future_speculation
    return spec_vector

# Function to execute trades based on LLM recommendation
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['NVDA'] = portfolio['positions'].get('NVDA', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'NVDA' in portfolio['positions'] and portfolio['positions']['NVDA'] > 0:
            shares_to_sell = portfolio['positions']['NVDA']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['NVDA'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")

# Function to calculate portfolio value
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value

# Fetch historical stock data for NVDA using yfinance
stock_data = yf.download('NVDA', start='2023-01-01', end='2023-12-31')

# Ensure the data is sorted by date and reset index
stock_data = stock_data.sort_index().reset_index()

# Calculate indicators
stock_data = calculate_indicators(stock_data)

# Loop over each day in the stock data
for index, row in stock_data.iterrows():
    if index < 50:  # Ensure enough data points for indicators
        continue
    
    date_str = row['Date'].strftime('%Y-%m-%d')
    current_price = row['Close']
    
    # Get recent indicator data as a vector
    indicators = stock_data.loc[index, ['MA20', 'MA50', 'BB_upper', 'BB_lower', 'Momentum']].values
    spec_vector = speculative_decoding(stock_data.iloc[:index])
    
    # Prepare prompt with both speculative and indicator vectors
    prompt = f"""
You are a financial analyst. Given the following speculative and indicator vectors for Nvidia (NVDA), determine if a 'buy', 'hold', or 'sell' strategy is most viable today. Respond with only one word: buy, hold, or sell.

Date: {date_str}
Speculative Vector: {spec_vector}
Indicator Vector (MA20, MA50, BB_upper, BB_lower, Momentum): {indicators}

Recommendation:
"""
    
    # Call the LLM service
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-70b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
    
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        execute_trade(recommendation, current_price, date_str)
        time.sleep(0.1)  # Simulate delay
    else:
        print(f"Error {response.status_code}: {response.text}")
        break

# Final portfolio value calculation
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

# Display final portfolio performance
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,
    'positions': {},
    'transaction_history': []
}

# Function to calculate indicators for quantitative analysis
def calculate_indicators(stock_data):
    stock_data['MA20'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['BB_upper'] = stock_data['MA20'] + (stock_data['Close'].rolling(window=20).std() * 2)
    stock_data['BB_lower'] = stock_data['MA20'] - (stock_data['Close'].rolling(window=20).std() * 2)
    stock_data['Momentum'] = stock_data['Close'] - stock_data['Close'].shift(4)
    return stock_data

# Speculative Decoding
def speculative_decoding(stock_data, lookback=10):
    recent_prices = stock_data['Close'].tail(lookback).values
    future_speculation = np.diff(recent_prices)
    spec_vector = future_speculation / np.linalg.norm(future_speculation) if np.linalg.norm(future_speculation) != 0 else future_speculation
    return spec_vector

# Execute Trades
def execute_trade(stock, recommendation, current_price, date):
    if recommendation == 'buy':
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions'][stock] = portfolio['positions'].get(stock, 0) + shares_to_buy
        portfolio['transaction_history'].append({'date': date, 'stock': stock, 'action': 'buy', 'shares': shares_to_buy, 'price': current_price, 'total': investment_amount})
        print(f"Bought {shares_to_buy:.2f} shares of {stock} at ${current_price:.2f} on {date}.")
    elif recommendation == 'sell' and stock in portfolio['positions']:
        shares_to_sell = portfolio['positions'][stock]
        proceeds = shares_to_sell * current_price
        portfolio['cash'] += proceeds
        portfolio['positions'][stock] = 0
        portfolio['transaction_history'].append({'date': date, 'stock': stock, 'action': 'sell', 'shares': shares_to_sell, 'price': current_price, 'total': proceeds})
        print(f"Sold {shares_to_sell:.2f} shares of {stock} at ${current_price:.2f} on {date}.")
    else:
        print(f"Holding {stock} position on {date}.")

# Calculate Portfolio Value
def calculate_portfolio_value(stock_prices):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * stock_prices[stock]
    return total_value

# Fetch Historical Stock Data for Multiple Stocks
stocks = ['NVDA', 'AAPL', 'MSFT']
stock_data = {ticker: yf.download(ticker, start='2023-01-01', end='2023-12-31').sort_index().reset_index() for ticker in stocks}

# Calculate Indicators
for ticker in stocks:
    stock_data[ticker] = calculate_indicators(stock_data[ticker])

# Loop Over Each Day and Each Stock
for ticker in stocks:
    for index, row in stock_data[ticker].iterrows():
        if index < 50:
            continue
        date_str = row['Date'].strftime('%Y-%m-%d')
        current_price = row['Close']
        indicators = stock_data[ticker].loc[index, ['MA20', 'MA50', 'BB_upper', 'BB_lower', 'Momentum']].values
        spec_vector = speculative_decoding(stock_data[ticker].iloc[:index])
        
        # Prepare Transaction History Summary
        history_summary = "\n".join([f"{trans['date']}: {trans['action']} {trans['shares']:.2f} shares of {trans['stock']} at ${trans['price']:.2f}" 
                                     for trans in portfolio['transaction_history'][-30:]])

        # Prompt for the LLM
        prompt = f"""
You are a financial analyst with access to a history of 30 previous transactions and a snapshot of the current portfolio. Based on this, provide a 'buy', 'hold', or 'sell' recommendation for each stock today, with an aim to maximize returns based on current holdings, market indicators, and cash balance.

Date: {date_str}
Stock: {ticker}
Current Portfolio:
- Cash: ${portfolio['cash']:.2f}
- Positions: {portfolio['positions']}
Speculative Vector: {spec_vector}
Indicator Vector (MA20, MA50, BB_upper, BB_lower, Momentum): {indicators}

Recent Transaction History:
{history_summary}

Respond with only one word: buy, hold, or sell.
"""

        # Call LLM Service
        data = {
            "messages": [{"role": "user", "content": prompt}],
            "model": "llama3.1-8b",
            "service_name": "cerebras",
            "temperature": 0.7,
            "max_tokens": 50,
            "top_p": 0.9,
        }
        response = requests.post(llm_url, json=data, headers=llm_headers)
        
        if response.status_code == 200:
            result = response.json()
            recommendation = result["content"].strip().lower()
            if recommendation in ['buy', 'hold', 'sell']:
                print(f"Date: {date_str}, Stock: {ticker}, LLM Recommendation: {recommendation}")
                execute_trade(ticker, recommendation, current_price, date_str)
            else:
                print(f"Invalid recommendation '{recommendation}' for {ticker} on {date_str}.")
            time.sleep(0.1)
        else:
            print(f"Error {response.status_code}: {response.text}")
            break

# Final Portfolio Calculation
final_prices = {ticker: stock_data[ticker].iloc[-1]['Close'] for ticker in stocks}
final_portfolio_value = calculate_portfolio_value(final_prices)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,
    'positions': {},
    'transaction_history': []
}

# Function to calculate indicators for quantitative analysis
def calculate_indicators(stock_data):
    stock_data['MA20'] = stock_data['Close'].rolling(window=20).mean()
    stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['BB_upper'] = stock_data['MA20'] + (stock_data['Close'].rolling(window=20).std() * 2)
    stock_data['BB_lower'] = stock_data['MA20'] - (stock_data['Close'].rolling(window=20).std() * 2)
    stock_data['Momentum'] = stock_data['Close'] - stock_data['Close'].shift(4)
    return stock_data

# Speculative Decoding
def speculative_decoding(stock_data, lookback=10):
    recent_prices = stock_data['Close'].tail(lookback).values
    future_speculation = np.diff(recent_prices)
    spec_vector = future_speculation / np.linalg.norm(future_speculation) if np.linalg.norm(future_speculation) != 0 else future_speculation
    return spec_vector

# Execute Trades
def execute_trade(stock, recommendation, current_price, date):
    if recommendation == 'buy':
        target_allocation = 0.1
        if stock not in portfolio['positions']:
            investment_amount = portfolio['cash'] * target_allocation
        else:
            investment_amount = min(portfolio['cash'] * 0.05, portfolio['cash'])
        
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions'][stock] = portfolio['positions'].get(stock, 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'stock': stock,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares of {stock} at ${current_price:.2f} on {date}.")
    elif recommendation == 'sell' and stock in portfolio['positions']:
        shares_to_sell = portfolio['positions'][stock]
        proceeds = shares_to_sell * current_price
        portfolio['cash'] += proceeds
        portfolio['positions'][stock] = 0
        portfolio['transaction_history'].append({
            'date': date,
            'stock': stock,
            'action': 'sell',
            'shares': shares_to_sell,
            'price': current_price,
            'total': proceeds
        })
        print(f"Sold {shares_to_sell:.2f} shares of {stock} at ${current_price:.2f} on {date}.")
    else:
        print(f"Holding {stock} position on {date}.")

# Calculate Portfolio Value
def calculate_portfolio_value(stock_prices):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * stock_prices[stock]
    return total_value

# Calculate Allocation of Each Stock in Portfolio
def calculate_allocation(current_prices):
    total_value = calculate_portfolio_value(current_prices)
    allocation = {}
    for stock, shares in portfolio['positions'].items():
        allocation[stock] = (shares * current_prices[stock]) / total_value if total_value > 0 else 0
    return allocation

# Fetch Historical Stock Data for Multiple Stocks
stocks = ['NVDA', 'AAPL', 'MSFT']
stock_data = {ticker: yf.download(ticker, start='2023-01-01', end='2023-12-31').sort_index().reset_index() for ticker in stocks}

# Calculate Indicators
for ticker in stocks:
    stock_data[ticker] = calculate_indicators(stock_data[ticker])

# Loop Over Each Day and Each Stock
for ticker in stocks:
    for index, row in stock_data[ticker].iterrows():
        if index < 50:
            continue
        date_str = row['Date'].strftime('%Y-%m-%d')
        current_price = row['Close']
        indicators = stock_data[ticker].loc[index, ['MA20', 'MA50', 'BB_upper', 'BB_lower', 'Momentum']].values
        spec_vector = speculative_decoding(stock_data[ticker].iloc[:index])
        
        # Calculate Allocation and Prepare Transaction History Summary
        current_prices = {ticker: stock_data[ticker].iloc[-1]['Close'] for ticker in stocks}
        allocation = calculate_allocation(current_prices)
        history_summary = "\n".join([f"{trans['date']}: {trans['action']} {trans['shares']:.2f} shares of {trans['stock']} at ${trans['price']:.2f}" 
                                     for trans in portfolio['transaction_history'][-30:]])

        # Enhanced Prompt for the LLM
        prompt = f"""
You are a financial analyst managing a portfolio across multiple stocks. Aim to maximize returns while diversifying the portfolio based on current holdings, allocation percentage, recent transaction history, and market indicators. Provide a single-word recommendation for each stock today.

Date: {date_str}
Stock: {ticker}
Current Portfolio:
- Cash: ${portfolio['cash']:.2f}
- Positions: {portfolio['positions']}
- Allocation: {allocation}
Speculative Vector: {spec_vector}
Indicator Vector (MA20, MA50, BB_upper, BB_lower, Momentum): {indicators}

Recent Transaction History:
{history_summary}

Respond with only one word—either "buy", "hold", or "sell".
"""

        # Call LLM Service
        data = {
            "messages": [{"role": "user", "content": prompt}],
            "model": "llama3.1-8b",
            "service_name": "cerebras",
            "temperature": 0.1,  # Lower temperature for more consistent and focused response
            "max_tokens": 2,     # Restrict tokens to enforce brevity
            "top_p": 0.9,
        }
        response = requests.post(llm_url, json=data, headers=llm_headers)
        
        if response.status_code == 200:
            result = response.json()
            recommendation = result["content"].strip().lower()
            if recommendation in ['buy', 'hold', 'sell']:
                print(f"Date: {date_str}, Stock: {ticker}, LLM Recommendation: {recommendation}")
                execute_trade(ticker, recommendation, current_price, date_str)
            else:
                print(f"Invalid recommendation '{recommendation}' for {ticker} on {date_str}.")
            time.sleep(0.1)
        else:
            print(f"Error {response.status_code}: {response.text}")
            break

# Final Portfolio Calculation
final_prices = {ticker: stock_data[ticker].iloc[-1]['Close'] for ticker in stocks}
final_portfolio_value = calculate_portfolio_value(final_prices)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)